# DNBLab Jupyter Notebook zur SRU Abfrage

## SRU - Schnittstellenabfrage und Datenauslieferung

Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage über die SRU-Schnittstelle mit Python. In dem Jupyter Notebook kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst die exemplarische Abfrage und Ausgabe der Daten im Format MARC21-xml zur weiteren Verarbeitung und den Export der Ergebnisse als CSV-Datei. 

In [1]:
import urllib.parse
from pyodide.http import open_url, pyfetch
from js import fetch
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

## Abfrage über alle Datensätze... <a class="anchor" id="Teil2"></a>

In [2]:
async def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    
    r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
    r_text = await r.text()
    xml = soup(r_text, features="xml")
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = await fetch(base_url + "?" + urllib.parse.urlencode(params))  
            r_text = await r.text()
            xml = soup(r_text, features="xml")
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

## Felder Titel und Links zu den Objekten durchsuchen... <a class="anchor" id="Teil3"></a>

In [ ]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # link
    link = xml.xpath("marc:datafield[@tag = '856']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        link = link[0].text
        #titel = unicodedata.normalize("NFC", titel)
    except:
        link = "unkown"
        
          # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        titel = titel[0].text
        #titel = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"
        
    
    meta_dict = {"idn":idn,
                 "titel":titel,
                 "link":link}
    
    return meta_dict

## Abfrage nach Suchwort "Pandemie" im Titel + Links zu frei verfügbaren Objekten!

In [ ]:
records = await dnb_sru('tit=Pandemie and location=onlinefree')
print(len(records), 'Ergebnisse')

## Anzeige der Treffer in einer Tabelle...  <a class="anchor" id="Teil4"></a>

In [ ]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

## Speichern der Ergebnisse als CSV-Datei!

In [18]:
df.to_csv("SRU_Titel.csv", index=False)